In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem as ch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoLarsCV, ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFECV, SelectFromModel
import pickle as pkl
import seaborn as sns

In [33]:
def split_data(X_new):
    X_final, validate = np.split(X_new.sample(frac=1), [int(.8*len(X_new))])
    X_train, X_test, y_train, y_test = train_test_split(X_final.drop(columns=['pChemBL'],inplace=False), X_final['pChemBL'], test_size=0.2, random_state=42)
    X_validate = validate.drop(columns='pChemBL',inplace=False).copy()
    y_validate = validate['pChemBL']
    return X_train, X_test, X_validate, y_train, y_test, y_validate
def train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate,estimators=150):
    model2 = RandomForestRegressor(n_estimators=estimators)
    model2.fit(X_train,y_train)
    r_square = model2.score(X_train,y_train)
    rmse = np.sqrt(mean_squared_error(model2.predict(X_train),y_train))
    print('Number of estimators = {}'.format(estimators))
    print("Training set results:\nRMSE = {}\t R^2 = {}".format(rmse,r_square))
    q_square = model2.score(X_test,y_test)
    rmse = np.sqrt(mean_squared_error(model2.predict(X_test),y_test))
    print("Test set results\nRMSE = {}\t Q^2 = {}".format(rmse,q_square))
    q_square = model2.score(X_validate,y_validate)
    rmse = np.sqrt(mean_squared_error(y_validate,model2.predict(X_validate)))
    print("Validation set results\nRMSE = {}\t Q^2 = {}".format(rmse,q_square))    
    return model2
def split_train(X,estimators=150):
    X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X)
    model = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate,estimators)
    return model
def get_cols(n):
    cols = []
    for i in range(n):
        cols.append('component'+str(i+1))
    return cols

In [3]:
X = pd.read_csv('no_zeros_no_chembl.csv')
y = X['pChemBL']
X.drop(columns='pChemBL',inplace=True)

In [5]:
reg = ExtraTreesRegressor(n_estimators=150)
reg = reg.fit(X, y)

In [ ]:
reg.score(X,y)

In [38]:
model = SelectFromModel(reg, prefit=True)
X_new = model.transform(X)
X_new.shape               


(2861, 279)

In [39]:
cols = []
for i in range(X_new.shape[1]):
    cols.append('comp'+str(i+1))
X_new = pd.DataFrame(data=X_new,columns=cols)

In [40]:
X_new['pChemBL'] = y
X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X_new)
model = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate)

Training set results:
RMSE = 0.2612342814942656	 R^2 = 0.951231462226615
Test set results
RMSE = 0.6711355166778638	 Q^2 = 0.6546759899619428
Validation set results
RMSE = 0.7139772320201792	 Q^2 = 0.5980952575220048


In [ ]:
model = SelectFromModel(reg, prefit=True,max_features=25)
X_new = model.transform(X)
cols = []
for i in range(X_new.shape[1]):
    cols.append('comp'+str(i+1))
X_new = pd.DataFrame(data=X_new,columns=cols)
X_new['pChemBL'] = y
X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X_new)
model = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate)

In [ ]:
X_new = SelectKBest(mutual_info_regression, k=50).fit_transform(X, y)

cols = []
for i in range(50):
    cols.append('comp'+str(i+1))
X_red = pd.DataFrame(data=X_new, columns=cols)

In [33]:
X_red['pChemBL'] = y
X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X_red)
RFR = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate)

Training set results:
RMSE = 0.2739322287215124	 R^2 = 0.9468244640055198
Test set results
RMSE = 0.6161916091171696	 Q^2 = 0.7009729340649702
Validation set results
RMSE = 0.6717152615156313	 Q^2 = 0.6397030563793147


In [41]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
ko = 15
X_new = SelectKBest(mutual_info_regression, k=ko).fit_transform(X, y)

cols = []
for i in range(ko):
    cols.append('comp'+str(i+1))
X_red = pd.DataFrame(data=X_new, columns=cols)
X_red['pChemBL'] = y
X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X_red)
RFR = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate)

Training set results:
RMSE = 0.2835793651287902	 R^2 = 0.9410312390992518
Test set results
RMSE = 0.6982204063849987	 Q^2 = 0.6342943954025049
Validation set results
RMSE = 0.7285472901564857	 Q^2 = 0.6090261968313742


In [6]:
from sklearn.feature_selection import RFECV
select = RFECV(reg,step =200,n_jobs=-1,min_features_to_select= 50,verbose=1)
select.fit(X,y)
X_new = select.transform(X)

Fitting estimator with 1208 features.
Fitting estimator with 1008 features.
Fitting estimator with 808 features.


In [10]:
cols = []
for i in range(X_new.shape[1]):
    cols.append('comp'+str(i+1))
X_red = pd.DataFrame(data=X_new, columns=cols)
X_red['pChemBL'] = y
X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X_red)
RFR = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate)

Training set results:
RMSE = 0.26038643091486924	 R^2 = 0.9506658314566686
Test set results
RMSE = 0.6892970161527208	 Q^2 = 0.6311874891385347
Validation set results
RMSE = 0.7153796013506576	 Q^2 = 0.624192456639348


In [21]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
#X_red.drop(columns='pChemBL',inplace=True)
X_red.head()
X_red.drop(columns='comp50',inplace=True)

In [ ]:
cols = []
for i in range(50):
    cols.append('comp'+str(i+1))
X_new = SelectKBest(mutual_info_regression, k=50).fit_transform(X_red, y)
X_red = pd.DataFrame(data=X_new, columns=cols)

In [22]:
X_red['pChemBL'] = y
X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X_red)
RFR = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate)

Training set results:
RMSE = 0.25776643379761544	 R^2 = 0.9506208177595016
Test set results
RMSE = 0.6484785432255382	 Q^2 = 0.6888873283556138
Validation set results
RMSE = 0.6953832131541388	 Q^2 = 0.6545151705403043


In [25]:
cols=[]
for i in range(25):
    cols.append('comp'+str(i+1))
X_red.drop(columns='pChemBL',inplace=True)    
X_new = SelectKBest(mutual_info_regression, k=25).fit_transform(X_red, y)
X_red = pd.DataFrame(data=X_new, columns=cols)
X_red['pChemBL'] = y
X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X_red)
RFR = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate)

Training set results:
RMSE = 0.2769289461389025	 R^2 = 0.9442085567589638
Test set results
RMSE = 0.7155156440570092	 Q^2 = 0.6291208962414494
Validation set results
RMSE = 0.6721193455727127	 Q^2 = 0.6449727110165622


<h4> For display/rerun(the best model)

In [70]:
X = pd.read_csv('no_zeros_no_chembl.csv')
y = X['pChemBL']
X.drop(columns='pChemBL',inplace=True)
selections = SelectKBest(mutual_info_regression, k=50)
X_new = selections.fit_transform(X, y)
cols = []
for i in range(50):
    cols.append('comp'+str(i+1))
X_red = pd.DataFrame(data=X_new, columns=cols)
X_red['pChemBL'] = y

In [10]:
X_train, X_test, X_validate, y_train, y_test, y_validate = split_data(X_red)
RFR = train_RFR(X_train, X_test, X_validate, y_train, y_test, y_validate)

Training set results:
RMSE = 0.2683276019767677	 R^2 = 0.9467019849913931
Test set results
RMSE = 0.7066172160204794	 Q^2 = 0.6299947142698358
Validation set results
RMSE = 0.610511017731687	 Q^2 = 0.7289860444995122


In [74]:
cols = selections.get_support(indices=True)
good_columns = []
all_columns = X.columns
for i in range(len(all_columns)):
    if i in cols:
        good_columns.append(X.columns[i])
print(good_columns)

['naAromAtom', 'nAromBond', 'BCUTc-1l', 'BCUTc-1h', 'BCUTp-1h', 'SpMax1_Bhm', 'SpMin1_Bhm', 'SpMin3_Bhm', 'SpMax1_Bhv', 'SpMax2_Bhv', 'SpMin1_Bhv', 'SpMin7_Bhv', 'SpMax1_Bhe', 'SpMax3_Bhe', 'SpMin1_Bhe', 'SpMin2_Bhe', 'SpMin4_Bhe', 'SpMin6_Bhe', 'SpMin8_Bhe', 'SpMax1_Bhp', 'SpMax2_Bhp', 'SpMax3_Bhp', 'SpMin1_Bhp', 'SpMax1_Bhi', 'SpMin1_Bhi', 'SpMin2_Bhi', 'C2SP2', 'SCH-6', 'VCH-5', 'SpMAD_Dt', 'SHaaCH', 'SaaN', 'minHBa', 'maxaaN', 'HybRatio', 'nAtomP', 'MDEN-22', 'MDEN-23', 'MLFER_A', 'MLFER_BH', 'piPC5', 'piPC7', 'piPC8', 'piPC9', 'piPC10', 'TpiPC', 'R_TpiPCTPC', 'nTRing', 'WTPT-4', 'WTPT-5']


In [75]:
len(good_columns)

50

<h3>SVR

In [6]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf',verbose=True,tol=0.001)
svr.fit(X_red.drop(columns='pChemBL'),y)
svr.score(X_red.drop(columns='pChemBL'),y)

[LibSVM]

0.3415220888673084

<h3>Elastic Net</h3>

In [15]:
from sklearn.linear_model import ElasticNetCV
e_net = ElasticNetCV(cv=5, random_state=0,normalize=True,verbose=1,n_jobs=-1,selection='random')
e_net.fit(X,y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    6.9s remaining:   10.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.0s finished


ElasticNetCV(cv=5, n_jobs=-1, normalize=True, random_state=0,
             selection='random', verbose=1)

In [16]:
e_net.score(X,y)

0.4269174010118165

In [38]:
e_net.l1_ratio

0.5

In [20]:
select = SelectFromModel(e_net,max_features=50,prefit=True)
Xnet = select.transform(X)
Xnet = pd.DataFrame(data=Xnet,columns=get_cols(50))
lol = Xnet.copy()
lol['pChemBL'] = y

In [27]:
RFR = split_train(lol)

Training set results:
RMSE = 0.2921107380950572	 R^2 = 0.9368475422256108
Test set results
RMSE = 0.7575229122063979	 Q^2 = 0.6162990062778935
Validation set results
RMSE = 0.7332009809977204	 Q^2 = 0.5763856120934157


In [29]:
RFR = split_train(lol,125)

Training set results:
RMSE = 0.28630673474348817	 R^2 = 0.9404941110581913
Test set results
RMSE = 0.7225212294716707	 Q^2 = 0.6023557995383836
Validation set results
RMSE = 0.7376490014209028	 Q^2 = 0.5913700576341223


<h3>Lasso LARS CV</h3>

In [39]:
X['pChemBL'] = y
cond_no = np.linalg.cond(X)
print("condition number of X = {}".format(cond_no))
X.drop(columns='pChemBL',inplace=True)

condition number of X = 7.4725446142904415e+37


In [40]:
X.head()

nAcid    ALogP      ALogp2       AMR        apol  naAromAtom  nAromBond  \
0      0  -0.7109    0.505379   70.4932   73.378583          16         18   
1      0  -1.4578    2.125181   44.0418   64.304653          18         20   
2      0   0.0748    0.005595   27.2874   69.268860          27         29   
3      0 -10.0458  100.918098  183.2952  142.418303           6          6   
4      0  -1.7897    3.203026   32.3564   58.685481          21         23   

   nAtom  nHeavyAtom  nH  ...       AMW      WTPT-1    WTPT-2     WTPT-3  \
0     65          34  31  ...  7.157502   70.514766  2.073964  27.938805   
1     52          31  21  ...  7.830281   65.167083  2.102164  17.955201   
2     57          37  20  ...  8.827511   76.232497  2.060338  31.795843   
3    135          64  71  ...  6.678230  125.580825  1.962200  61.611922   
4     47          30  17  ...  8.491716   62.402934  2.080098  16.872064   

      WTPT-4     WTPT-5  WPATH  WPOL  XLogP  Zagreb  
0  11.267053  16.671753   3404    59  2.188     184  
1   2.565237  15.389964   2593    55  6.575     180  
2   2.556014  22.098211   4979    56  5.589     198  
3  20.276064  41.335857  21440    92 -0.194     302  
4  13.655728   3.216336   2319    52  4.844     168  

[5 rows x 1208 columns]

In [63]:
from sklearn.linear_model import LassoLarsCV
LL = LassoLarsCV(verbose=1,n_jobs=-1)
LL.fit(X,y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.6s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 40 iterations, i.e. alpha=1.606e-03, with an active set of 32 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 98 iterations, i.e. alpha=7.512e-04, with an active set of 66 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\HP\Anaconda3\lib\site-packages\skle

LassoLarsCV(n_jobs=-1, verbose=1)

In [64]:
LL.score(X,y)

0.508183761099205

In [52]:
np.finfo(np.float).eps*10

2.220446049250313e-15

In [65]:
select = SelectFromModel(LL,max_features=50,prefit=True)
Xnet = select.transform(X)
Xnet = pd.DataFrame(data=Xnet,columns=get_cols(50))
lol = Xnet.copy()
lol['pChemBL'] = y

In [69]:
RFR = split_train(lol,75)

Number of estimators = 75
Training set results:
RMSE = 0.28237345738837427	 R^2 = 0.9398527351582295
Test set results
RMSE = 0.7304957501713284	 Q^2 = 0.6146137535875539
Validation set results
RMSE = 0.7289022337701816	 Q^2 = 0.6308351286764
